In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# # import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [45]:
# dataset_path = 'allUsers.lcl.csv'
dataset_path = "../data/tommaso_data/allUsers.lcl.csv"

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)



In [52]:
from sklearn.metrics import accuracy_score, f1_score

def save_results(clf, X_test, y_test, nclf, dataf):
    dataf.loc[nclf, 'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    # dataf.loc[nclf, 'test f1 score (0)'] = f1_score(y_test, clf.predict(X_test), pos_label=0)
    # dataf.loc[nclf, 'test f1 score (1)'] = f1_score(y_test, clf.predict(X_test), pos_label=1)
    dataf.loc[nclf, 'test f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='macro')
    # df.loc[nclf,'ROC AUC'] = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    return dataf

results_df = pd.DataFrame()



# KNN

# Class recognition with User label

In [26]:
df2 = df.drop(['Class'], axis=1)
labels_Class = df['Class']


X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)


In [27]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [28]:
print(knn.score(X_test, y_test))


0.9602432778489116


# Hand owner recognition with Class label

In [29]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [30]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [31]:
print(knn.score(X_test, y_test))

0.8646606914212548


# Class recognition without User label

In [32]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

In [33]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9596670934699104


# Class recognition without Class label

In [34]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)

labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [35]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.8646606914212548


# SVM

In [36]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pip insall sklearn.grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)

df = df.replace('?', np.nan)
df = df.dropna(thresh=len(df) * 0.8 , axis=1)
df = df.replace(np.nan, 0)

drop_indices = np.random.choice(df.index, 75000, replace=False)
df = df.drop(drop_indices)
df

ERROR: unknown command "insall" - maybe you meant "install"


,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5
33,1,0,57.188299,71.969974,-61.055158,60.051138,10.398977,-70.538890,32.442550,80.991365,-57.096942,84.6529482275572,68.0469160650217,-74.055015896666,75.7646115464786,42.5284198448163,-73.7535957212517,0,0,0
78,1,0,63.062767,10.947741,-67.376305,32.352293,80.960891,-57.529997,78.165618,42.775898,-71.338002,57.4873324616827,72.5611548843288,-60.9611183408917,87.8151137077638,68.2230424290105,-70.7850157933284,0,0,0
87,1,0,61.934470,10.727026,-68.611776,33.526941,80.985220,-56.033271,86.182511,67.885419,-72.569835,76.2973935785158,42.3892300154246,-73.2856585667272,0,0,0,0,0,0
90,1,0,60.180069,10.325333,-70.370529,33.186244,80.794394,-56.559278,85.466683,67.695669,-73.352786,76.0301174866085,42.3258183901331,-73.6275735151941,0,0,0,0,0,0
129,1,0,35.182310,82.087600,-53.818944,61.517994,11.588590,-68.744317,77.472179,43.453587,-71.541039,87.1941939182506,68.8185456397786,-70.7798921893051,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77985,5,14,80.593767,29.334851,-50.360564,-1.814506,99.163432,6.324889,27.792626,136.313769,-33.078748,-1.94192191497827,124.998316380621,-42.6902787229678,-29.9077194692271,78.2695941493792,-16.8258669259183,-21.8078039197755,104.540374004303,-54.2342889366686
78028,5,14,80.916157,28.187499,-52.610947,61.793672,101.641934,-6.187505,-0.156308,96.674074,0.406627,86.7309957725075,61.3187288821744,-59.7945148152106,-30.61846828279,78.4682435346278,-15.9722981120188,-22.7877171799973,104.742327465586,-53.2960917882787
78064,5,14,88.760663,59.931435,-63.671039,-2.155721,125.068618,-41.709071,77.992760,32.445566,-44.458812,-0.970094534882875,97.628680322975,2.98397838329817,55.1306108005195,129.367411656344,-43.3191719272339,-30.185045982206,78.1287444666287,-16.3108442369111
78070,5,14,55.548078,129.512357,-41.791831,27.242311,136.745154,-31.614706,-30.276580,78.726291,-15.758450,78.1881695610848,32.7732849048862,-43.5608262729756,-1.80177811487272,98.3460107051729,4.87855892136542,28.2541676428054,107.390859619162,10.6446104793982


# Class recognition without User label

In [37]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.25483870967741934


# SVC gread search class recognition without user label

In [38]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.4s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.2s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [39]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.8693548387096774
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


# Hand owner recognition without Class label


In [40]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.1693548387096774


# Gread SearchHand owner recognition without Class label

In [41]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.2s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.5s


KeyboardInterrupt: 

In [ ]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

# Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [46]:
# dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


# Class recognition with User label

In [47]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [48]:
clf = RandomForestClassifier(max_depth=200, random_state=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9863636363636363

In [53]:
results_df = save_results(clf, X_test, y_test, 'Random forest', results_df)



# Gread Search

## Show results:

In [54]:
results_df.sort_values(by=['test acc'], ascending=False)



,test acc,test f1 score (W)
Random forest,0.986364,0.940244
